In [48]:
from importlib import reload
import honours_project.simulation as simulation
reload(simulation)
import honours_project.data as data
reload(data)

import json

In [ ]:
from honours_project.data import PNE, Cycle, SimulationResult

# Example of problematic seed
sim = simulation.Simulation(5, 2, seed=1737044418)
result1 = sim.run()
result2 = sim.run()
print(json.dumps(result1, indent=4, cls=data.EnhancedJSONEncoder))
print(json.dumps(result2, indent=4, cls=data.EnhancedJSONEncoder))

{
    "time": 0.06784283299930394,
    "iteration": 91
}
{
    "time": 0.02161129200248979,
    "iteration": 35,
    "allocations": [
        {
            "bidder": 3,
            "auction": 0,
            "price": 0.42134011961374696
        },
        {
            "bidder": 2,
            "auction": 1,
            "price": 0.2694658968969589
        }
    ]
}
